In [ ]:
# for generating the best pattern by logistic regression

In [ ]:
import mne
from matplotlib import pyplot as plt
plt.style.use(['default'])
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pickle
import os
import time
import math
import random
import argparse
import copy
import scipy
import scipy.io as sio
from scipy import signal
import mat73
import sklearn
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import seaborn as sns
# from pytopo import topoplotIndie

In [5]:
group_name='PD';
band_name='delta';

In [6]:
group_name, band_name

('PD', 'delta')

In [8]:
score = sio.loadmat(group_name+'_score.mat')
score = score[band_name]
pattern = sio.loadmat(group_name+'_PC_'+band_name+'_pattern.mat')
GMP_ref = sio.loadmat(group_name+'_GMP_ref.mat')
SRP = sio.loadmat(group_name+'_SRP.mat')

In [11]:
indX = np.concatenate((np.ones(12),np.zeros(12))) #1111...0000, PD n=12, HC n=12
len(indX), indX

(24,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]))

In [12]:
# all PC combinations
from itertools import combinations

nums = [0,1,2,3,4]
combi = list(combinations(nums, 1)) + list(combinations(nums, 2)) + list(combinations(nums, 3)) + list(combinations(nums, 4)) + list(combinations(nums, 5))

In [13]:
combi

[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4),
 (0, 1, 2, 3),
 (0, 1, 2, 4),
 (0, 1, 3, 4),
 (0, 2, 3, 4),
 (1, 2, 3, 4),
 (0, 1, 2, 3, 4)]

In [14]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay

In [15]:
# pred = subject * PC score
# 0번째 조합에 대해서 multiple linear regression
# 나중에 for 문 돌려야 함
AIC_list = []
AUC_list = []
ACC_list = []
gmeans_list = []
sensitivity_list = []
specificity_list = []

for num in range(len(combi)):
    pred = score[:,combi[num]]
    clf = LogisticRegression(random_state=0).fit(pred, indX)

    'AIC calculation'
    BX= clf.intercept_[0] + pred@clf.coef_.T
    log_likelihood=np.sum(indX* np.squeeze(BX)- np.squeeze(np.log(1+np.exp(BX))))
    aic = -2*log_likelihood+2*(len(combi[num])+1)
    AIC_list.append(aic)

    y_pred=clf.predict(pred)
    y_score = clf.predict_proba(pred)

    cm = sklearn.metrics.confusion_matrix(indX, y_pred)
    conf_matrix = pd.DataFrame(data = cm,
                               columns = ['Predicted:HC', 'Predicted:'+group_name],
                               index =['HC', group_name])
    'for plot and summary'
    # Accuracy, AUC
    fpr, tpr, thresholds = roc_curve(indX, clf.predict_proba(pred)[:, 1], pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)
    ACC_list.append(clf.score(pred, indX))
    AUC_list.append(roc_auc)

    gmeans = np.sqrt(tpr * (1-fpr)) # sqrt(Sensitivity * Specificity)
    maxidx  = np.argmax(gmeans)
    gmeans_list.append(gmeans[maxidx])
    best_threshold = thresholds[maxidx]
    sensitivity_list.append(tpr[maxidx])
    specificity_list.append((1-fpr)[maxidx])
    
    PC_name_for_plot = []
    for i in combi[num]:
        PC_name_for_plot.append(i+1)

## find best combination of PCs

In [16]:
np.argmax(AUC_list), np.argmin(AIC_list), np.argmax(ACC_list), np.argmax(gmeans_list)

(26, 17, 17, 17)

In [17]:
'1: AUC, 2:AIC, 3:ACC, 4:gmeans'
criteria = 2

if criteria == 1: #AUC
    num = np.argmax(AUC_list)
elif criteria == 2: #AIC
    num = np.argmin(AIC_list)
elif criteria == 3: #ACC
    num = np.argmax(ACC_list)
elif criteria == 4: #gmeans
    num = np.argmax(gmeans_list)

In [ ]:
pred = score[:,combi[num]]
clf = LogisticRegression(random_state=0).fit(pred, indX)

'AIC calculation'
BX= clf.intercept_[0] + pred@clf.coef_.T
log_likelihood=np.sum(indX* np.squeeze(BX)- np.squeeze(np.log(1+np.exp(BX))))
aic = -2*log_likelihood+2*(len(combi[num])+1)

y_pred=clf.predict(pred)
y_score = clf.predict_proba(pred)

cm = sklearn.metrics.confusion_matrix(indX, y_pred)
conf_matrix = pd.DataFrame(data = cm,
                           columns = ['Predicted:HC', 'Predicted:'+group_name],
                           index =['HC', group_name])
'for plot and summary'
sns.heatmap(conf_matrix, annot = True, fmt = 'd', cmap = "Blues")

# Accuracy, AUC
fpr, tpr, thresholds = roc_curve(indX, clf.predict_proba(pred)[:, 1], pos_label=1)
roc_auc = metrics.auc(fpr, tpr)

gmeans = np.sqrt(tpr * (1-fpr)) # sqrt(Sensitivity * Specificity)
maxidx  = np.argmax(gmeans)
best_threshold = thresholds[maxidx]

PC_name_for_plot = []
for i in combi[num]:
    PC_name_for_plot.append(i+1)

'for roc plot'
plt.figure()
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, 
    estimator_name='LR of PC '+ str(PC_name_for_plot))
display.plot()
plt.ylabel('TPR (sensitivity)')
plt.xlabel('FPR (1-specificity)')
plt.scatter(fpr[maxidx], tpr[maxidx], marker='o', 
            color='black', label='Best threshold: p= '
            +str(round(best_threshold,4)))
plt.legend()

print('Combination: ', PC_name_for_plot)
print('\ncoef, intercept')
print(clf.coef_, clf.intercept_)
print (classification_report(indX, y_pred))
print('\nAIC: ', round(aic,4), '/ ACC: ',round(clf.score(pred, indX), 4), '/ AUC: ',round(roc_auc, 4))
print('\nBest Threshold=%.4f, G-Mean=%.4f' % (thresholds[maxidx], gmeans[maxidx]))
print('sensitivity, specificity', round(tpr[maxidx],4),round((1-fpr)[maxidx],4))

In [19]:
# best combination:  [1, 2, 5]
pattern_whole = np.squeeze((pattern['PC1'], pattern['PC2'], pattern['PC3'], pattern['PC4'], pattern['PC5'])) #(5 PCs, 15002 voxels)
pattern_selected = pattern_whole[combi[num], :] # pattern generated from best combination of PCs (PC1, PC2, PC5)

In [20]:
np.shape(clf.coef_), np.shape(pattern_selected)

((1, 3), (3, 15002))

In [21]:
# final pattern : sum(each coef * each PC pattern)
GIS_A = clf.coef_ @ pattern_selected

# z transformed pattern
GIS_Az = (GIS_A-GIS_A.mean())/ GIS_A.std()

# original subject score for 12 PD and 12 HC
subject_score = SRP['SRP_'+band_name]@ GIS_Az.T # ((24, 1) = (24, 15002) @ (15002, 1))

In [25]:
np.shape(subject_score), np.shape(SRP['SRP_'+band_name]), np.shape(GIS_Az.T)

((24, 1), (24, 15002), (15002, 1))

In [26]:
HCd = subject_score[len(eval(group_name)):]
RBDd = subject_score[:len(eval(group_name))]

# z transformed (reference to HC) original subject score
HCdz = (HCd-HCd.mean())/HCd.std()
RBDdz = (RBDd-HCd.mean())/HCd.std()

In [27]:
subj_z = np.concatenate((HCdz, RBDdz), axis=0) # HC, PD

In [28]:
'result_'+group_name+'_'+band_name

'result_PD_delta'

In [35]:
HC_mean_std=(HCd.mean(),HCd.std())

In [36]:
result = {}

result['GIS_A'] = GIS_A

result['GIS_Az']=GIS_Az

result['subj_z']=subj_z

result['HC_mean_std']=HC_mean_std

sio.savemat('result_'+group_name+'_'+band_name+'.mat', result)